In [13]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

driver = webdriver.Chrome("chromedriver.exe")

driver.get('http://www.findpeoplesearch.com/')
 # click Expand Form button
# the [0] is there because find_elements_by_id returns a list
expand_form_button = driver.find_elements_by_id("more-btn")[0]
expand_form_button.click()

In [14]:
# <input type="text" name="full_name" id="full_name"
driver.wait = WebDriverWait(driver, 10)
full_name_tbox = driver.wait.until(EC.presence_of_element_located((By.NAME, "full_name")))
full_name_tbox.send_keys('Marty Owings')

In [15]:
#<button type="submit" class="btn btn-success btn-lg" id="button-search" style="width: 75%">Search</button>
search_button = driver.find_element_by_id("button-search")
search_button.send_keys(Keys.ENTER)
# wait for  <div class="alert alert-info col-md-8"><h4>Search Results for...</h4>
start_time = time.time()
driver.wait = WebDriverWait(driver, 20)
try:
    page_loaded = driver.wait.until(EC.presence_of_element_located((By.ID, "new-search2")))
except TimeoutException:
    try:
        soup = BeautifulSoup(driver.page_source, "html5lib")
        no_results = soup.findAll("div", class_="panel panel-primary")
        print("No results found")
        driver.quit()
    except NoSuchElementException:
        print("Something went wrong.")
        driver.quit()   
    
    
end_time = time.time()
print('Elapsed time = '+ str(end_time - start_time))

No results found
Elapsed time = 23.90494132041931


In [ ]:
soup = BeautifulSoup(driver.page_source, "html5lib")
soups = []
soups.append(soup)
more_pages = True
while (more_pages):

    next_page = driver.find_element_by_xpath('//*[@id="search-results"]/div[4]/div/nav/ul/li[5]/a')
    next_page.send_keys(Keys.ENTER)
    start_time = time.time()
    driver.wait = WebDriverWait(driver, 20)
    page_loaded = driver.wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="search-results"]/div[4]/div/nav/ul/li[5]/a')))
    end_time = time.time()
    print('Elapsed time = '+ str(end_time - start_time))
    soup = BeautifulSoup(driver.page_source, "html5lib")
    soups.append(soup)
    try:
        last_page = driver.find_element_by_css_selector('li.next-page.disabled')
        #search-results > div:nth-child(6) > div > nav > ul > li.next-page.disabled
        more_pages = False
        break
    except NoSuchElementException:    
        more_pages = True
print (str(len(soups)) + ' pages scraped')

In [ ]:

# get all col-md-4 from all pages (from each soup in soups)
people = []
for soup in soups:
#    cols=soup.findAll("div",class_="col-md-4")
    cols=soup.findAll("div", class_="panel panel-primary")
    for i in range (0, len(cols)):
        people.append(cols[i])
print('number of persons found = ' + str(len(people)))

#todo loop through people[] and get data:
for z in range(0, len(people)):
    one_person = people[z]
    tags = one_person.findAll('li')

    data_headers = one_person.findAll('span', class_='data_header')
    data_header_names = []
    for i in range(0, len(data_headers)):
        data_header_names.append(data_headers[i].text.split())
    print (data_header_names) 
    num_of_addresses = 0
    num_of_phones = 0
    num_of_emails = 0
    for i in range (0, len(data_header_names)):
        if data_header_names[i][1] == 'Street':
            num_of_addresses = int(data_header_names[i][0])
        elif data_header_names[i][1] == 'Phone':
            num_of_phones = int(data_header_names[i][0])
        elif data_header_names[i][1] == 'Email':
            num_of_emails = int(data_header_names[i][0])

    print('Addresses=' + str(num_of_addresses) + '  Phones=' + str(num_of_phones) + '  Emails=' + str(num_of_emails))    

    for i in range(0, num_of_addresses):
        addy_data = str(tags[i])
        begin = addy_data.find('data-person="')+13
        end = addy_data.find('" data-person-address')
        addy_data_1 = addy_data[begin:end]
        print (addy_data_1)

    for j in range(num_of_addresses, num_of_addresses + num_of_phones):
        phone_data = str(tags[j])
        begin = phone_data.find('<b>Phone number: </b>') + 21
        end = phone_data.find('<br/><b>Company:')
        phone_data_1 = phone_data[begin:end]
        print (phone_data_1)

    for j in range(num_of_addresses + num_of_phones, num_of_addresses + num_of_phones + num_of_emails):
        email_data = str(tags[j])
        begin = email_data.find('<a href="mailto:') + 17
        end = email_data.find('">')
        email_data_1 = email_data[begin:end]
        print (email_data_1)